In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
import re
import requests
import math
from bs4 import BeautifulSoup
import matplotlib as mlp
import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg
import matplotlib.figure
import seaborn as sb
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
#CARS PRICES SCRAPING FROM WEB PAGE LISTINGS

In [23]:
fst_page_url = 'https://www.truecar.com/used-cars-for-sale/listings/'
url_requested = requests.get(fst_page_url)
fst_page_soup = BeautifulSoup(url_requested.content, 'lxml')
fst_page_soup

<!DOCTYPE html>
<html lang="en-US">
<head>
<title data-rh="true">Used Cars for Sale | TrueCar</title>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" data-rh="true" name="viewport"/><meta content="Search over 903,507 used Cars. TrueCar has over 1,443,366 listings nationwide, updated daily. Come find a great deal on used Cars in your area today!" data-rh="true" name="description"/>
<link data-qa="MasterHelmet" data-rh="true" href="https://listings-prod.tcimg.net/listings/" rel="preconnect"/><link data-qa="MasterHelmet" data-rh="true" href="https://static.tcimg.net" rel="preconnect"/><link data-qa="MasterHelmet" data-rh="true" href="https://consumer.tcimg.net/assets" rel="preconnect"/><link data-rh="true" href="https://static.tcimg.net/pac/7/7ec678517aba12406d2c80bd5064492171ac8f86.ico" rel="icon" type="image/x-icon"/><link data-rh="true" href="https://www.truecar.com/used-cars-for-sale/listings/" rel="canonical"/><link data-rh="true" href="https://

In [3]:
fst_page_prices = fst_page_soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
fst_page_prices

[<h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$24,479</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$14,590</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$18,810</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$7,495</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$26,500</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$28,300</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$8,495</h4>,
 <h4 class="heading-3 margin-y-1 font-weight-bold" data-qa="Heading" data-test="vehicleCardPricingBlockPrice">$12,971</h4>,
 <h4 class

In [4]:
re.findall('[0-9]+,[0-9]+', str(fst_page_prices))

['24,479',
 '14,590',
 '18,810',
 '7,495',
 '26,500',
 '28,300',
 '8,495',
 '12,971',
 '26,419',
 '10,995',
 '8,799',
 '29,940',
 '27,888',
 '12,924',
 '24,995',
 '7,944',
 '31,945',
 '9,991',
 '14,999',
 '7,991',
 '8,995',
 '3,990',
 '12,785',
 '7,889',
 '9,977',
 '24,888',
 '15,754',
 '10,099',
 '21,997',
 '10,987',
 '8,900',
 '30,989',
 '25,279']

In [ ]:
#ANOTHER WAY TO SCRAPE THE PRICES
#listings_soup = BeautifulSoup(requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=2').content, 'lxml')
#listings_soup.find_all('h4', {'class': 'heading-3 margin-y-1 font-weight-bold'})

In [5]:
# FUNCTION TO GET THE RAW HTML SOUPS FROM ALL 600 WEB PAGES

def get_soups(website_number):
    get_url = requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(website_number))
    return BeautifulSoup(get_url.content, 'lxml')

In [34]:
soups = list(map(get_soups, list(range(1, 10))))

In [7]:
# FUNCTION TO SCRAPE THE CARS PRICES FOR EACH WEB PAGE

def pricesscraper(soup):
    nth_page_prices_soup = soup.find_all('h4', {'data-test': 'vehicleCardPricingBlockPrice'})
    nth_page_prices = re.findall('[0-9]+,[0-9]+', str(nth_page_prices_soup))
    return nth_page_prices

In [12]:
prices = list(map(pricesscraper, soups))
prices

[['20,200',
  '12,999',
  '33,580',
  '7,495',
  '26,500',
  '28,300',
  '8,495',
  '12,971',
  '26,419',
  '10,995',
  '8,799',
  '29,940',
  '27,888',
  '12,924',
  '24,995',
  '7,944',
  '31,945',
  '9,991',
  '14,999',
  '7,991',
  '8,995',
  '3,990',
  '12,785',
  '7,889',
  '9,977',
  '24,888',
  '15,754',
  '10,099',
  '21,997',
  '10,987',
  '8,900',
  '30,989',
  '25,279'],
 ['10,488',
  '9,000',
  '11,995',
  '25,235',
  '26,988',
  '32,899',
  '32,000',
  '33,000',
  '9,900',
  '27,550',
  '33,191',
  '27,950',
  '19,991',
  '11,500',
  '6,915',
  '7,350',
  '8,981',
  '8,995',
  '6,995',
  '9,993',
  '10,998',
  '11,995',
  '23,590',
  '9,500',
  '13,000',
  '4,995',
  '18,991',
  '4,900',
  '9,891',
  '34,980'],
 ['12,897',
  '16,990',
  '5,995',
  '16,695',
  '23,977',
  '9,250',
  '12,995',
  '3,999',
  '9,991',
  '8,500',
  '17,909',
  '12,194',
  '9,985',
  '28,000',
  '1,995',
  '23,995',
  '6,985',
  '14,788',
  '5,995',
  '5,500',
  '9,200',
  '7,985',
  '3,495',
  

In [13]:
prices = str(prices).replace('[', '').replace(']', '').split(', ')
prices

["'20,200'",
 "'12,999'",
 "'33,580'",
 "'7,495'",
 "'26,500'",
 "'28,300'",
 "'8,495'",
 "'12,971'",
 "'26,419'",
 "'10,995'",
 "'8,799'",
 "'29,940'",
 "'27,888'",
 "'12,924'",
 "'24,995'",
 "'7,944'",
 "'31,945'",
 "'9,991'",
 "'14,999'",
 "'7,991'",
 "'8,995'",
 "'3,990'",
 "'12,785'",
 "'7,889'",
 "'9,977'",
 "'24,888'",
 "'15,754'",
 "'10,099'",
 "'21,997'",
 "'10,987'",
 "'8,900'",
 "'30,989'",
 "'25,279'",
 "'10,488'",
 "'9,000'",
 "'11,995'",
 "'25,235'",
 "'26,988'",
 "'32,899'",
 "'32,000'",
 "'33,000'",
 "'9,900'",
 "'27,550'",
 "'33,191'",
 "'27,950'",
 "'19,991'",
 "'11,500'",
 "'6,915'",
 "'7,350'",
 "'8,981'",
 "'8,995'",
 "'6,995'",
 "'9,993'",
 "'10,998'",
 "'11,995'",
 "'23,590'",
 "'9,500'",
 "'13,000'",
 "'4,995'",
 "'18,991'",
 "'4,900'",
 "'9,891'",
 "'34,980'",
 "'12,897'",
 "'16,990'",
 "'5,995'",
 "'16,695'",
 "'23,977'",
 "'9,250'",
 "'12,995'",
 "'3,999'",
 "'9,991'",
 "'8,500'",
 "'17,909'",
 "'12,194'",
 "'9,985'",
 "'28,000'",
 "'1,995'",
 "'23,995'",
 "'

In [14]:
prices = list(map(lambda x: x[1:-1], prices))
print(len(prices))
prices

2523


['20,200',
 '12,999',
 '33,580',
 '7,495',
 '26,500',
 '28,300',
 '8,495',
 '12,971',
 '26,419',
 '10,995',
 '8,799',
 '29,940',
 '27,888',
 '12,924',
 '24,995',
 '7,944',
 '31,945',
 '9,991',
 '14,999',
 '7,991',
 '8,995',
 '3,990',
 '12,785',
 '7,889',
 '9,977',
 '24,888',
 '15,754',
 '10,099',
 '21,997',
 '10,987',
 '8,900',
 '30,989',
 '25,279',
 '10,488',
 '9,000',
 '11,995',
 '25,235',
 '26,988',
 '32,899',
 '32,000',
 '33,000',
 '9,900',
 '27,550',
 '33,191',
 '27,950',
 '19,991',
 '11,500',
 '6,915',
 '7,350',
 '8,981',
 '8,995',
 '6,995',
 '9,993',
 '10,998',
 '11,995',
 '23,590',
 '9,500',
 '13,000',
 '4,995',
 '18,991',
 '4,900',
 '9,891',
 '34,980',
 '12,897',
 '16,990',
 '5,995',
 '16,695',
 '23,977',
 '9,250',
 '12,995',
 '3,999',
 '9,991',
 '8,500',
 '17,909',
 '12,194',
 '9,985',
 '28,000',
 '1,995',
 '23,995',
 '6,985',
 '14,788',
 '5,995',
 '5,500',
 '9,200',
 '7,985',
 '3,495',
 '5,499',
 '26,795',
 '14,419',
 '26,995',
 '10,999',
 '6,995',
 '3,500',
 '18,911',
 '16,

In [ ]:
#CARS YEARS SCRAPING FROM WEB PAGE LISTINGS

In [24]:
fst_page_years_soup = fst_page_soup.find_all('span', {'class': 'vehicle-card-year'})
fst_page_years_soup

In [25]:
re.findall('[12][0-9]{3}', str(fst_page_years_soup))

['2018',
 '2018',
 '2015',
 '2017',
 '2018',
 '2018',
 '2017',
 '2017',
 '2016',
 '2017',
 '2017',
 '2018',
 '2017',
 '2017',
 '2017',
 '2017',
 '2018',
 '2018',
 '2017',
 '2019',
 '2013',
 '2008',
 '2016',
 '2018',
 '2017',
 '2018',
 '2019',
 '2017',
 '2016',
 '2017',
 '2017',
 '2019',
 '2018']

In [26]:
# FUNCTION TO SCRAPE A FEATURE FROM EACH SOUP AND RETURN THE FEATURES LIST

def scraper(tag, element, element_description, regex):
    def featuresscraper(soup):
        nth_page_features_soup = soup.find_all(tag, {element: element_description})
        nth_page_features = re.findall(regex, str(nth_page_features_soup))
        return nth_page_features
    features = list(map(featuresscraper, soups))
    features = str(features).replace('[', '').replace(']', '').split(', ')
    features = list(map(lambda x: x[1: -1], features))
    return features

In [27]:
years = scraper('span', 'class', 'vehicle-card-year', '[12][0-9]{3}')
print(len(years))
years

2523


['2016',
 '2011',
 '2018',
 '2017',
 '2018',
 '2018',
 '2017',
 '2017',
 '2016',
 '2017',
 '2017',
 '2018',
 '2017',
 '2017',
 '2017',
 '2017',
 '2018',
 '2018',
 '2017',
 '2019',
 '2013',
 '2008',
 '2016',
 '2018',
 '2017',
 '2018',
 '2019',
 '2017',
 '2016',
 '2017',
 '2017',
 '2019',
 '2018',
 '2018',
 '2017',
 '2017',
 '2016',
 '2017',
 '2019',
 '2016',
 '2004',
 '2017',
 '2018',
 '2019',
 '2018',
 '2019',
 '2017',
 '2017',
 '2006',
 '2017',
 '2017',
 '2000',
 '2014',
 '2019',
 '2016',
 '2016',
 '2017',
 '2017',
 '2013',
 '2019',
 '2014',
 '2016',
 '2019',
 '2019',
 '2019',
 '2010',
 '2017',
 '2017',
 '2018',
 '2017',
 '2008',
 '2018',
 '2017',
 '2017',
 '2019',
 '2017',
 '2018',
 '2002',
 '2016',
 '2017',
 '2018',
 '2014',
 '2003',
 '2017',
 '2017',
 '2005',
 '2015',
 '2017',
 '2017',
 '2018',
 '2015',
 '2015',
 '2011',
 '2008',
 '2009',
 '2018',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2019',
 '2019',
 '2010',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2017',
 '2018',
 

In [28]:
# CARS MILEAGES SCRAPING FROM WEB PAGE LISTINGS

In [29]:
fst_page_mileages_soup = fst_page_soup.find_all('div', {'class': 'font-size-1 text-truncate'})
fst_page_mileages_soup

[<div class="font-size-1 text-truncate" data-test="vehicleCardTrim">LT with 1SD Hatchback Automatic</div>,
 <div class="font-size-1 text-truncate">$<!-- -->293<!-- --> <!-- -->off<!-- --> <!-- -->avg. list price</div>,
 <div class="font-size-1 text-truncate" data-test="vehicleMileage"><svg class="icon icon-before vehicle-card-icon icon-color-default" data-qa="IconSpeedometer" style="width:16px;height:16px;stroke-width:1.5" viewbox="0 0 24 24"><path d="M.28 15.67l2.68.19M1.04 12.26l2.59 1M2.77 9.22l2.24 1.76M5.32 6.82l1.57 2.26M8.47 5.28l3.21 10.58M11.92 4.74l.02 2.9M15.38 5.23l-.83 2.79M18.55 6.73l-1.62 2.41M21.13 9.09l-2.24 1.81M22.91 12.1l-2.64 1.06M23.72 15.5l-2.8.24"></path><circle cx="11.92" cy="16.64" r=".87"></circle></svg>31,269<!-- --> miles</div>,
 <div class="font-size-1 text-truncate" data-test="vehicleCardTrim">LX FWD CVT</div>,
 <div class="font-size-1 text-truncate">$<!-- -->15<!-- --> <!-- -->off<!-- --> <!-- -->avg. list price</div>,
 <div class="font-size-1 text-trunc

In [32]:
re.findall('g>[0-9]*,[0-9]+', str(fst_page_mileages_soup))

33

In [35]:
mileages_unf = scraper('div', 'class', 'font-size-1 text-truncate', 'g>[0-9]*,[0-9]+')
mileages = list(map(lambda mileage: mileage[2:], mileages_unf))
print(len(mileages))
mileages

272


['36,637',
 '48,455',
 '16,664',
 '152,738',
 '43,503',
 '15,681',
 '117,592',
 '45,776',
 '47,692',
 '71,593',
 '99,515',
 '41,244',
 '46,020',
 '34,896',
 '80,026',
 '44,453',
 '30,924',
 '79,030',
 '115,998',
 '21,019',
 '91,571',
 '133,000',
 '66,090',
 '53,254',
 '54,006',
 '40,562',
 '20,590',
 '64,490',
 '103,293',
 '60,229',
 '37,726',
 '25,234',
 '34,148',
 '135,297',
 '143,192',
 '92,832',
 '43,682',
 '46,922',
 '14,131',
 '31,445',
 '81,601',
 '77,855',
 '33,645',
 '13,795',
 '33,147',
 '23,225',
 '20,038',
 '178,978',
 '111,357',
 '106,832',
 '325,045',
 '79,611',
 '202,222',
 '62,430',
 '87,692',
 '85,730',
 '26,350',
 '31,624',
 '95,148',
 '104,691',
 '19,617',
 '31,303',
 '31,256',
 '107,220',
 '77,242',
 '31,980',
 '103,690',
 '32,776',
 '140,000',
 '84,861',
 '133,972',
 '29,981',
 '33,784',
 '102,068',
 '43,578',
 '233,189',
 '44,398',
 '73,617',
 '124,959',
 '143,106',
 '86,733',
 '51,556',
 '46,295',
 '199,066',
 '139,161',
 '75,325',
 '118,467',
 '7,668',
 '102,700

In [ ]:
#CARS LOCATIONS STATES SCRAPING FROM WEB PAGE LISTINGS

In [38]:
fst_page_states_soup = fst_page_soup.find_all('div', {'data-test': 'vehicleCardLocation'})
fst_page_states_soup

[<div class="vehicle-card-location font-size-1 margin-top-1" data-qa="Location" data-test="vehicleCardLocation"><svg class="icon icon-before vehicle-card-icon icon-color-default" data-qa="IconLocationPin" style="width:16px;height:16px;stroke-width:1.5" viewbox="0 0 24 24"><path d="M18.91 12.33L12.62 23 6.23 12.2"></path><path d="M6.44 12.55a7.5 7.5 0 0 1-1.35-3.93v-.39a7.54 7.54 0 0 1 15.08 0v.69a7.5 7.5 0 0 1-1.37 3.68"></path><circle cx="12.57" cy="8.09" r="2.7"></circle></svg>Columbia<!-- -->, <!-- -->SC</div>,
 <div class="vehicle-card-location font-size-1 margin-top-1" data-qa="Location" data-test="vehicleCardLocation"><svg class="icon icon-before vehicle-card-icon icon-color-default" data-qa="IconLocationPin" style="width:16px;height:16px;stroke-width:1.5" viewbox="0 0 24 24"><path d="M18.91 12.33L12.62 23 6.23 12.2"></path><path d="M6.44 12.55a7.5 7.5 0 0 1-1.35-3.93v-.39a7.54 7.54 0 0 1 15.08 0v.69a7.5 7.5 0 0 1-1.37 3.68"></path><circle cx="12.57" cy="8.09" r="2.7"></circle></

In [41]:
re.findall('[A-W][A-Y]', str(fst_page_states_soup))

['SC',
 'FL',
 'NC',
 'FL',
 'CO',
 'NC',
 'VA',
 'IL',
 'NH',
 'GA',
 'NC',
 'NE',
 'CA',
 'NC',
 'OH',
 'FL',
 'WA',
 'FL',
 'FL',
 'FL',
 'GA',
 'CA',
 'FL',
 'FL',
 'SC',
 'CA',
 'LA',
 'NC',
 'CA',
 'TX',
 'IN',
 'TX']

In [42]:
states = scraper('div', 'data-test', 'vehicleCardLocation', '[A-Z]{2}')
print(len(states))
states

273


['NC',
 'CA',
 'AZ',
 'FL',
 'CO',
 'NC',
 'VA',
 'IL',
 'NH',
 'GA',
 'NC',
 'NE',
 'CA',
 'NC',
 'OH',
 'FL',
 'WA',
 'FL',
 'FL',
 'FL',
 'GA',
 'CA',
 'FL',
 'FL',
 'SC',
 'CA',
 'LA',
 'NC',
 'CA',
 'TX',
 'AZ',
 'IN',
 'TX',
 'MO',
 'TX',
 'MD',
 'OH',
 'ID',
 'OK',
 'CA',
 'TX',
 'FL',
 'ND',
 'KY',
 'ND',
 'TX',
 'IL',
 'IN',
 'CA',
 'CA',
 'PA',
 'IL',
 'CA',
 'TX',
 'NC',
 'SC',
 'FL',
 'TX',
 'PA',
 'TX',
 'IN',
 'TX',
 'KY',
 'OR',
 'VA',
 'MN',
 'CO',
 'WA',
 'AL',
 'FL',
 'TX',
 'IL',
 'FL',
 'GA',
 'KS',
 'NJ',
 'CA',
 'GA',
 'TN',
 'TX',
 'KY',
 'GA',
 'WA',
 'NY',
 'TN',
 'MN',
 'IL',
 'CA',
 'GA',
 'WA',
 'TX',
 'OK',
 'WA',
 'LA',
 'VA',
 'GA',
 'CA',
 'NJ',
 'FL',
 'FL',
 'GA',
 'MD',
 'WA',
 'FL',
 'OR',
 'MT',
 'IN',
 'TX',
 'MI',
 'VT',
 'CA',
 'GA',
 'FL',
 'TX',
 'WA',
 'UT',
 'NY',
 'TN',
 'CA',
 'TX',
 'CA',
 'SC',
 'MO',
 'CA',
 'FL',
 'NC',
 'PA',
 'FL',
 'VA',
 'SC',
 'OR',
 'VA',
 'GA',
 'CO',
 'MD',
 'VA',
 'VA',
 'FL',
 'UT',
 'VA',
 'IN',
 'TX',
 'NC',

In [ ]:
#CARS LOCATIONS CITIES SCRAPING FROM WEB PAGE LISTINGS

In [43]:
fst_page_cities_soup = fst_page_soup.find_all('div', {'data-test': 'vehicleCardLocation'})
fst_page_cities_soup[0]

<div class="vehicle-card-location font-size-1 margin-top-1" data-qa="Location" data-test="vehicleCardLocation"><svg class="icon icon-before vehicle-card-icon icon-color-default" data-qa="IconLocationPin" style="width:16px;height:16px;stroke-width:1.5" viewbox="0 0 24 24"><path d="M18.91 12.33L12.62 23 6.23 12.2"></path><path d="M6.44 12.55a7.5 7.5 0 0 1-1.35-3.93v-.39a7.54 7.54 0 0 1 15.08 0v.69a7.5 7.5 0 0 1-1.37 3.68"></path><circle cx="12.57" cy="8.09" r="2.7"></circle></svg>Columbia<!-- -->, <!-- -->SC</div>

In [44]:
re.findall('[A-Z][a-z]+[. ]*[A-Z]*[a-z]*[. ]*[A-Z]*[a-z]*', str(fst_page_cities_soup))

['Location',
 'CardLocation',
 'IconLocationPin',
 'Columbia',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Ocala',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Wilson',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'St. Petersburg',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Boulder',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Lumberton',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Chesapeake',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Chicago',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Gorham',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Savannah',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Denver',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Blair',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Hanford',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Shelby',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Hamilton',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Homosassa',

In [46]:
cities_unf = scraper('div', 'data-test', 'vehicleCardLocation', '[A-Z][a-z]+[. ]*[A-Z]*[a-z]*[. ]*[A-Z]*[a-z]*')
print(len(cities_unf))
cities_unf

1092


['Location',
 'CardLocation',
 'IconLocationPin',
 'Cary',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Norco',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Tempe',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'St. Petersburg',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Boulder',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Lumberton',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Chesapeake',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Chicago',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Gorham',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Savannah',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Denver',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Blair',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Hanford',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Shelby',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Hamilton',
 'Location',
 'CardLocation',
 'IconLocationPin',
 'Homosassa',
 'Lo

In [47]:
cities = [city for city in cities_unf if cities_unf.index(city) in list(range(3, len(cities_unf), 4))]
print(len(cities))
cities

273


['Cary',
 'Norco',
 'Tempe',
 'St. Petersburg',
 'Boulder',
 'Lumberton',
 'Chesapeake',
 'Chicago',
 'Gorham',
 'Savannah',
 'Denver',
 'Blair',
 'Hanford',
 'Shelby',
 'Hamilton',
 'Homosassa',
 'Puyallup',
 'Hollywood',
 'Margate',
 'Hollywood',
 'Savannah',
 'El Cajon',
 'Plantation',
 'Miami',
 'Rock Hill',
 'Antioch',
 'Metairie',
 'Charlotte',
 'Sacramento',
 'Austin',
 'Phoenix',
 'Bloomington',
 'Houston',
 'Belton',
 'Austin',
 'Aberdeen',
 'Delaware',
 'Boise',
 'Broken Arrow',
 'Modesto',
 'Spring',
 'Fort Myers',
 'Minot',
 'Columbia',
 'Minot',
 'Desoto',
 'McCook',
 'Jeffersonville',
 'Moreno Valley',
 'Fontana',
 'West Chester',
 'Chicago',
 'Sacramento',
 'Blue Mound',
 'Wilson',
 'Florence',
 'Tampa',
 'Desoto',
 'Walnutport',
 'Houston',
 'Greenwood',
 'Dallas',
 'Henderson',
 'Medford',
 'Winchester',
 'Shakopee',
 'Greeley',
 'Wenatchee',
 'Birmingham',
 'Hollywood',
 'San Antonio',
 'Chicago',
 'Tampa',
 'Duluth',
 'Olathe',
 'Haskell',
 'Fontana',
 'Duluth',
 'Je

In [ ]:
#CARS EXTERIOR COLORS SCRAPING FROM WEP PAGE LISTINGS

In [48]:
fst_page_colors_soup = fst_page_soup.find_all('div', {'data-test': 'vehicleCardColors'})
fst_page_colors_soup[0]

<div class="vehicle-card-location font-size-1 margin-top-1 text-truncate" data-qa="ExteriorInteriorColor" data-test="vehicleCardColors"><svg class="icon icon-before vehicle-card-icon icon-color-default" data-qa="IconPaintBucket" style="width:16px;height:16px;stroke-width:1.5" viewbox="0 0 24 24"><path d="M18.45 18.94l2.13-3.61 2.16 3.65"></path><path d="M22.67 18.86a2.54 2.54 0 0 1 .46 1.33v.13a2.55 2.55 0 1 1-5.1 0v-.23a2.54 2.54 0 0 1 .46-1.24"></path><circle cx="14.5" cy="9.5" r="1"></circle><path d="M20.42 12.26L10.33 22.35 1.3 13.31 11.39 3.22M14.5 9V1M10.64 2.09l10.84 10.84"></path></svg>Black<!-- --> exterior, <!-- -->Black<!-- --> interior</div>

In [49]:
re.findall('->[A-Z][a-z]+', str(fst_page_colors_soup))

['->Black',
 '->Black',
 '->Black',
 '->Black',
 '->Gray',
 '->Black',
 '->Black',
 '->Black',
 '->Gray',
 '->Gray',
 '->Black',
 '->Unknown',
 '->Gray',
 '->Beige',
 '->Gray',
 '->Gray',
 '->Blue',
 '->Gray',
 '->Gray',
 '->Black',
 '->Unknown',
 '->Black',
 '->Gray',
 '->Black',
 '->Black',
 '->Unknown',
 '->Black',
 '->Black',
 '->Gray',
 '->Black',
 '->Black',
 '->Black',
 '->Gray']

In [50]:
exterior_colors_unf = scraper('div', 'data-test', 'vehicleCardColors', 'g>[A-Z][a-z]+')
len(exterior_colors_unf)

273

In [51]:
exterior_colors = list(map(lambda color: color[2:], exterior_colors_unf))
print(len(exterior_colors))
exterior_colors

273


['Silver',
 'Silver',
 'Red',
 'Gray',
 'White',
 'Silver',
 'Gray',
 'Blue',
 'Black',
 'White',
 'Black',
 'Gray',
 'White',
 'Black',
 'Silver',
 'Blue',
 'Silver',
 'White',
 'Black',
 'Black',
 'Gray',
 'Black',
 'Gray',
 'Black',
 'Gray',
 'Unknown',
 'Silver',
 'Black',
 'White',
 'Black',
 'Black',
 'Black',
 'Black',
 'Silver',
 'Silver',
 'Gray',
 'White',
 'White',
 'Black',
 'White',
 'Black',
 'Gray',
 'Silver',
 'White',
 'Silver',
 'White',
 'Gold',
 'Black',
 'Silver',
 'Black',
 'Silver',
 'Black',
 'White',
 'Red',
 'White',
 'Red',
 'White',
 'White',
 'White',
 'White',
 'White',
 'Silver',
 'White',
 'White',
 'White',
 'Red',
 'Black',
 'Black',
 'Black',
 'Silver',
 'Red',
 'White',
 'Gray',
 'Black',
 'Gray',
 'Silver',
 'White',
 'Blue',
 'Gray',
 'White',
 'White',
 'Gray',
 'Silver',
 'Red',
 'Black',
 'White',
 'Gray',
 'White',
 'White',
 'Black',
 'Silver',
 'White',
 'White',
 'Gray',
 'Black',
 'Silver',
 'White',
 'Red',
 'Silver',
 'White',
 'White',
 

In [ ]:
#CARS INTERIOR COLORS SCRAPING FROM WEB PAGE LISTINGS

In [52]:
re.findall('->[A-Z][a-z]+', str(fst_page_colors_soup))

['->Black',
 '->Black',
 '->Black',
 '->Black',
 '->Gray',
 '->Black',
 '->Black',
 '->Black',
 '->Gray',
 '->Gray',
 '->Black',
 '->Unknown',
 '->Gray',
 '->Beige',
 '->Gray',
 '->Gray',
 '->Blue',
 '->Gray',
 '->Gray',
 '->Black',
 '->Unknown',
 '->Black',
 '->Gray',
 '->Black',
 '->Black',
 '->Unknown',
 '->Black',
 '->Black',
 '->Gray',
 '->Black',
 '->Black',
 '->Black',
 '->Gray']

In [53]:
interior_colors_unf = scraper('div', 'data-test', 'vehicleCardColors', '->[A-Z][a-z]+')
interior_colors = list(map(lambda color: color[2:], interior_colors_unf))
print(len(interior_colors))
interior_colors

273


['Black',
 'Black',
 'Black',
 'Black',
 'Gray',
 'Black',
 'Black',
 'Black',
 'Gray',
 'Gray',
 'Black',
 'Unknown',
 'Gray',
 'Beige',
 'Gray',
 'Gray',
 'Blue',
 'Gray',
 'Gray',
 'Black',
 'Unknown',
 'Black',
 'Gray',
 'Black',
 'Black',
 'Unknown',
 'Black',
 'Black',
 'Gray',
 'Black',
 'Black',
 'Black',
 'Gray',
 'Black',
 'Unknown',
 'Black',
 'Gray',
 'Gray',
 'Beige',
 'Unknown',
 'Gray',
 'Gray',
 'Unknown',
 'Unknown',
 'Unknown',
 'Gray',
 'Black',
 'Beige',
 'Gray',
 'Gray',
 'Brown',
 'Black',
 'Gray',
 'Gray',
 'Black',
 'Black',
 'Black',
 'Gray',
 'Black',
 'Black',
 'Unknown',
 'Black',
 'Black',
 'Gray',
 'Black',
 'Black',
 'Unknown',
 'Black',
 'Unknown',
 'Black',
 'Gray',
 'Black',
 'Black',
 'Unknown',
 'Gray',
 'Beige',
 'Gray',
 'Black',
 'Black',
 'Beige',
 'Black',
 'Black',
 'Black',
 'Unknown',
 'Black',
 'Unknown',
 'Black',
 'Gray',
 'Gray',
 'Black',
 'Black',
 'Black',
 'Black',
 'Black',
 'Black',
 'Beige',
 'Unknown',
 'Black',
 'Black',
 'Unknow

In [ ]:
#CARS CONDITION (NUMBER OF ACCIDENTS) SCRAPING FROM WEB PAGE LISTINGS

In [54]:
fst_page_accidents_soup = fst_page_soup.find_all('div', {'data-test': 'vehicleCardCondition'})
fst_page_accidents_soup[0]

<div class="vehicle-card-location font-size-1 margin-top-1" data-qa="ConditionHistory" data-test="vehicleCardCondition"><svg class="icon icon-before vehicle-card-icon icon-color-default" data-qa="IconCarFront" style="width:16px;height:16px;stroke-width:1.5" viewbox="0 0 24 24"><path d="M22 13.13v6.23a.7.7 0 0 1-.68.72h-2.71a.69.69 0 0 1-.65-.72v-1.28H6v1.28a.69.69 0 0 1-.65.72H2.64a.7.7 0 0 1-.64-.72v-5.91a2.84 2.84 0 0 1 .34-1.35l2.09-4A3.76 3.76 0 0 1 7.6 6.08H17a3.79 3.79 0 0 1 3.4 2.47L21.75 12a2.93 2.93 0 0 1 .25 1.13z"></path><path d="M7.96 14.08h-3"></path><path d="M2.83 11.08H.46M23.46 11.08h-2.09M18.46 11.08h-13"></path><path d="M18.96 14.08h-3"></path></svg>No accidents, <!-- -->1 Owner, <!-- -->Personal use</div>

In [55]:
re.findall('[0-9]*[A-z]* accident[s]*', str(fst_page_accidents_soup))

['No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 '2 accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 '1 accident',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 '1 accident',
 'No accidents',
 '1 accident',
 'No accidents',
 'No accidents',
 '1 accident',
 '1 accident',
 '1 accident',
 'No accidents',
 '1 accident',
 'No accidents',
 'No accidents',
 '2 accidents',
 'No accidents',
 'No accidents',
 '1 accident']

In [56]:
accidents = scraper('div', 'data-test', 'vehicleCardCondition', '[0-9]*[A-z]* accident[s]*')
print(len(accidents))
accidents

273


['No accidents',
 '1 accident',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 '2 accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 '1 accident',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 '1 accident',
 'No accidents',
 '1 accident',
 'No accidents',
 'No accidents',
 '1 accident',
 '1 accident',
 '1 accident',
 'No accidents',
 '1 accident',
 'No accidents',
 'No accidents',
 '2 accidents',
 'No accidents',
 'No accidents',
 '1 accident',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 '1 accident',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 '1 accident',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No accidents',
 'No

In [ ]:
#CARS MAKE SCRAPING FROM WEB PAGES LISTINGS